In [1]:
import numpy as np
import libpysal as ps
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

/home/mjarumani/.local/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [16]:
data = pd.read_csv('combined_2016.csv')
data['ndvi'] = data['ndvi'] * 0.0001
data
#data=data.mask(data == '')

,Unnamed: 0,lon,lat,date,ndvi,sm,psum
0,0,39.410,3.271,20160101,0.3416,0.071,0.000
1,1,39.445,3.261,20160101,0.3416,0.085,0.000
2,2,39.492,3.254,20160101,0.3416,0.085,0.000
3,3,39.541,3.251,20160101,0.3159,0.156,0.000
4,4,39.590,3.253,20160101,0.3104,0.156,0.000
...,...,...,...,...,...,...,...
434143,434143,40.042,14.648,20161218,0.0758,0.058,7.835
434144,434144,40.091,14.650,20161218,0.0758,0.061,6.025
434145,434145,40.141,14.651,20161218,0.0811,0.061,6.025
434146,434146,40.112,14.676,20161218,0.0758,0.061,6.025


In [17]:
data['psum'].max()

320.588

In [18]:
data.replace(np.nan, 0, inplace=True)

In [19]:
import geopandas
gdf = geopandas.GeoDataFrame(
    data, geometry=geopandas.points_from_xy(data.lon, data.lat), crs="EPSG:4326"
)

print(gdf.head())

   Unnamed: 0     lon    lat      date    ndvi     sm  psum  \
0           0  39.410  3.271  20160101  0.3416  0.071   0.0   
1           1  39.445  3.261  20160101  0.3416  0.085   0.0   
2           2  39.492  3.254  20160101  0.3416  0.085   0.0   
3           3  39.541  3.251  20160101  0.3159  0.156   0.0   
4           4  39.590  3.253  20160101  0.3104  0.156   0.0   

                   geometry  
0  POINT (39.41000 3.27100)  
1  POINT (39.44500 3.26100)  
2  POINT (39.49200 3.25400)  
3  POINT (39.54100 3.25100)  
4  POINT (39.59000 3.25300)  


In [20]:
#Prepare dataset inputs
y = gdf['ndvi'].values.reshape((-1,1))
X = gdf[['sm','psum']].values
u = gdf['lon']
v = gdf['lat']
coords = list(zip(u,v))

In [21]:
np.random.seed(500000)
sample = np.random.choice(range(434148), 0)
mask = np.ones_like(y,dtype=bool).flatten()
mask[sample] = False

In [22]:
cal_coords = np.array(coords)[mask]
cal_y = y[mask]
cal_X = X[mask]

pred_coords = np.array(coords)[~mask]
pred_y = y[~mask]
pred_X = X[~mask]

In [ ]:
#Calibrate GWR model
gwr_selector = Sel_BW(cal_coords, cal_y, cal_X)
gwr_bw = gwr_selector.search()
#print(gwr_bw)
#model = GWR(cal_coords, cal_y, cal_X, gwr_bw)
#gwr_results = model.fit()

In [ ]:
print(gwr_bw)

In [23]:
index = np.arange(len(cal_y))
test = index[-434148:]
X_test = X[test]
coords_test = np.array(coords)[test]

In [24]:
gwr_bw = 139

In [ ]:
model = GWR(cal_coords, cal_y, cal_X, bw=gwr_bw, fixed=False, kernel='gaussian')
results = model.predict(coords_test, X_test)
print(results.params.shape)

In [26]:
results.predictions

array([[0.30577004],
       [0.31751864],
       [0.31047806],
       ...,
       [0.09231153],
       [0.09243417],
       [0.09241261]])

In [27]:
data['pred'] = results.predictions
data.head()

,Unnamed: 0,lon,lat,date,ndvi,sm,psum,pred
0,0,39.410,3.271,20160101,0.3416,0.071,0.0,0.305770
1,1,39.445,3.261,20160101,0.3416,0.085,0.0,0.317519
2,2,39.492,3.254,20160101,0.3416,0.085,0.0,0.310478
3,3,39.541,3.251,20160101,0.3159,0.156,0.0,0.341934
4,4,39.590,3.253,20160101,0.3104,0.156,0.0,0.328903


In [28]:
data['biom'] = ((6480.2 * data['pred']) - 958.6)/1000
data

,Unnamed: 0,lon,lat,date,ndvi,sm,psum,pred,biom
0,0,39.410,3.271,20160101,0.3416,0.071,0.000,0.305770,1.022851
1,1,39.445,3.261,20160101,0.3416,0.085,0.000,0.317519,1.098984
2,2,39.492,3.254,20160101,0.3416,0.085,0.000,0.310478,1.053360
3,3,39.541,3.251,20160101,0.3159,0.156,0.000,0.341934,1.257204
4,4,39.590,3.253,20160101,0.3104,0.156,0.000,0.328903,1.172760
...,...,...,...,...,...,...,...,...,...
434143,434143,40.042,14.648,20161218,0.0758,0.058,7.835,0.092189,-0.361196
434144,434144,40.091,14.650,20161218,0.0758,0.061,6.025,0.092039,-0.362167
434145,434145,40.141,14.651,20161218,0.0811,0.061,6.025,0.092312,-0.360403
434146,434146,40.112,14.676,20161218,0.0758,0.061,6.025,0.092434,-0.359608


In [29]:
data.to_csv('results-2016.csv')